In [ ]:
import pandas as pd
import numpy as np

# building_distance : 각 건물 사이 거리
building_distance = pd.read_csv('/content/건물별 이동시간 최종.csv')

# classroom : classroom_index	| building	| room |	max	| plugs(개수) |	ac_bool	| table(개인용=1,다인용=0)	| chair(일체=1,분리=0)
classroom = pd.read_csv('/content/강의실 건물별 TOP3 최종.csv')

# users : user_index |	building	| time	| plugs(필요도) |	table(개인용=1,다인용=0)	| chair(일체=1,분리=0)
users = pd.read_csv('/content/강의실 가상 사용자 5000명 최종.csv')


################ 강의실 true_label 함수 ####################
############################################################
def true_label_classroom(user):

  # 사용자 선택 랜드마크와 강의실 사이 거리 구하기  
  user_building = user['building']
  distance = building_distance[['building', user_building]].copy()
  distance.rename(columns = {user_building: 'distance'}, inplace = True)

  classroom_with_dis = pd.merge(classroom, distance, on= 'building', how ='left' )


  # 사용자 선호도 의자, 책상 만족시키는 강의실 찾기
  satisfy_user_idx = classroom_with_dis['table'] == user['table']
  desk_score = classroom_with_dis['table'].copy()
  desk_score[satisfy_user_idx] == 1
  desk_score[~satisfy_user_idx] == 0

  satisfy_user_idx = classroom_with_dis['chair'] == user['chair']
  chair_score = classroom_with_dis['chair'].copy()
  chair_score[satisfy_user_idx] == 1
  chair_score[~satisfy_user_idx] == 0


  # max min Normalization 사용 : (x-min(x) /(max(x)-min(x))
  plug_norm = ((classroom_with_dis['plugs']*user['plugs']) - (classroom_with_dis['plugs']*user['plugs']).min()) / ((classroom_with_dis['plugs']*user['plugs']).max()-(classroom_with_dis['plugs']*user['plugs']).min())
  distance_norm = (classroom_with_dis['distance'] - classroom_with_dis['distance'].min() ) / (classroom_with_dis['distance'].max() - classroom_with_dis['distance'].min())
  size_norm = (classroom_with_dis['max'] - classroom_with_dis['max'].min() ) / (classroom_with_dis['max'].max() - classroom_with_dis['max'].min())
  ac_norm = (classroom_with_dis['ac_bool'] - classroom_with_dis['ac_bool'].min() ) / (classroom_with_dis['ac_bool'].max() - classroom_with_dis['ac_bool'].min())
  desk_norm = (desk_score - desk_score.min() ) / (desk_score.max() - desk_score.min())
  chair_norm = (chair_score - chair_score.min() ) / (chair_score.max() - chair_score.min())


  # 사용자에 대한 선호도 norm DataFrame 생성  
  norm_col = {
        'classroom_index':classroom_with_dis['classroom_index'],
        'plug_norm':plug_norm*0.86,
        'distance_norm':distance_norm*(-5), 
        'size_norm': size_norm*1,
        'ac_norm':ac_norm*0.6,
        'table_norm' : desk_norm * 0.68,     
        'chair_norm' : chair_norm * 0.68
        }
  mmnorm = pd.DataFrame(norm_col)

  # 사용자 입장에서 각 강의실 점수 계산 + sort
  score = pd.concat([mmnorm['classroom_index'], mmnorm[['plug_norm','distance_norm','size_norm','ac_norm', 'table_norm', 'chair_norm']].sum(axis=1)], axis =1)
  score.columns=['classroom_index','score']


  # 사용자 점수 배열 생성
  classroom_score = score['score'].copy().tolist()
  
  return classroom_score
######################################################
######################################################



# 사용자 값 저장할 데이터프레임 생성
user_index = np.array(range(len(users))).tolist()
user_score = pd.DataFrame({'user_index': user_index})
scores = []

# 사용자 각각의 모든 강의실에 대한 점수(true label) 생성
for idx,row in users.iterrows():
  classroom_score = true_label_classroom(row)
  scores.append(classroom_score)

user_score['score'] = scores.copy()


In [ ]:
# 사용자 점수(true label) 저장
user_score.to_csv('5000명 사용자-강의실 점수 TL.csv', index=False)
user_score

,user_index,score
0,0,"[-1.428715403367442, -1.3147441343782433, -1.2..."
1,1,"[-1.249149155242221, -1.1351778862530222, -1.0..."
2,2,"[-1.7380564392982203, -1.624085170309021, -1.5..."
3,3,"[-1.018018390498776, -0.9040471215095774, -0.8..."
4,4,"[-0.7884499791121834, -0.6744787101229847, -0...."
...,...,...
4995,4995,"[-0.19732070269680313, -0.08334943370760461, -..."
4996,4996,"[0.9216404497534272, 1.0356117187426257, 1.098..."
4997,4997,"[1.8389883224289312, 1.95295959141813, 2.01560..."
4998,4998,"[1.3094035562413522, 1.423374825230551, 1.4860..."
